In [25]:
import gradio as gr
import requests
from langchain.llms import Ollama

# TMDB API 키
TMDB_KEY = "97f5ea5f12fa2ebcb7d31beb4ad328b0"

# Ollama 모델 사용할 때는 API 키 필요 없습니다
# Ollama 모델 초기화
model = Ollama(model="gemma2", base_url="http://localhost:11434")

# 1. TMDB API 요청 함수
# "TMDB 정보"를 받아옴
def flim(title):
    url = "https://api.themoviedb.org/3/search/movie"
    # 주소 뒤 파라미터
    params = {
        'api_key': TMDB_KEY,
        'query': title,
        'language': 'ko-KR'
    }

    try:
        #url 응답받기
        response = requests.get(url, params=params)
        
        # API 응답 상태 확인
        response.raise_for_status()  # HTTP 에러 발생 시 예외 발생
        
        # "json" data로 응답
        data = response.json()
                       # 에러를 방지해줍니다.
        results = data.get('results', [])
        print("결과:", results)
        print("결과2:", results[0])
        # 결과가 없는 경우 처리
        if not results:
            return f"'{title}'에 대한 영화를 찾을 수 없습니다. 다른 제목을 입력해 보세요."

        # 첫 번째 영화 "정보 추출"
        # "첫번째"만 가져옴 [0]
        movie = results[0]
        movie_info = f"""🎬 제목: {movie.get('title')}
        📅 개봉일: {movie.get('release_date')}
        ⭐ 평점: {movie.get('vote_average')}/10
        📖 줄거리: {movie.get('overview')}
        """

        # 포스터만 보여주면 됨
        poster_path = movie.get('poster_path')
        poster_url = f"https://image.tmdb.org/t/p/w500{poster_path}"
        # "https://image.tmdb.org/t/p/w500{poster_path}" : 포스터 경로 
        return movie_info, poster_url

    except requests.exceptions.RequestException as e:
        return f"API 요청 중 오류 발생: {str(e)}"
    except IndexError:
        return f"영화 정보를 처리하는 중 오류 발생: 검색 결과가 없습니다."

# 2. 챗봇 함수
def flim_chatbot(message):
    # 1. TMDB로부터 영화 정보 받아오기
    movie_info = flim(message)
    
    # 영화 정보를 찾지 못한 경우 그대로 반환
    if "찾을 수 없습니다" in movie_info or "오류 발생" in movie_info:
        return movie_info

    # 튜플 언패킹(Tuple Unpacking)
    # 분리
    movie_info, poster_url = movie_info

    # 2. AI 모델에 전달할 "프롬프트 구성과 이해"
    prompt = f"""당신은 친근하고 전문적인 영화 전문가 챗봇입니다. 
다음 영화 정보를 바탕으로 사용자에게 재미있고 유익한 정보를 제공해주세요.

영화 정보: {movie_info} 

사용자 질문: {message}

사용자들이 ~설명해주세요 혹은 ~알려주세요라고 해도 영화정보를 알려주시면 됩니다.
위 정보를 바탕으로 영화에 대해 친근하고 상세하게 설명해주세요."""

    try:
        # Ollama 모델 호출 
        # Ollama 모델에 전달하여 응답을 생성합니다. (invoke)
        response = model.invoke(prompt)
        return poster_url, response
    
    except Exception as e:
        return f"챗봇 응답 생성 중 오류 발생: {str(e)}"

# Gradio 인터페이스 생성
iface = gr.Interface(
    # 함수만 받습니다
    fn=flim_chatbot,
    # example
    inputs=gr.Textbox(lines=10, placeholder="영화 제목을 입력하세요 (예: 기생충)"),
    outputs=[gr.Image(label="포스터", show_label=True),  
             gr.Textbox(label="상세설명", lines=20)
                   
    ],
    title="🎬 영화 정보 챗봇",
    description="영화 제목을 입력하면 상세한 정보와 함께 AI가 영화에 대해 설명해드립니다!",
    # examples=[
    #     ["기생충"],
    #     ["어벤져스가 뭐야?"],
    #     ["타이타닉에 대해 알려줘"],
    #     ["인터스텔라는 재미있어?"],
    #     ["기생충이라는 영화 있어?"],
    #     ["탑건 설명해줘"]
    # ]
)




# TMDB API 요청 함수
def TV(title):
    url = "https://api.themoviedb.org/3/search/tv"
    # 주소 뒤 파라미터
    params = {
        'api_key': TMDB_KEY,
        'query': title,
        'language': 'ko-KR'
    }

    try:
        #url 응답받기
        response = requests.get(url, params=params)
        
        # API 응답 상태 확인
        response.raise_for_status()  # HTTP 에러 발생 시 예외 발생
        
        # json data로 응답
        data = response.json()
                       # 에러를 방지해줍니다.
        results = data.get('results', [])
        print("결과:", results)
        print("결과2:", results[0])
        # 결과가 없는 경우 처리
        if not results:
            return f"'{title}'에 대한 프로그램을 찾을 수 없습니다. 다른 제목을 입력해 보세요."

        # 첫 번째 영화 "정보 추출"
        # "첫번째"만 가져옴 [0]
        show = results[0]
        show_info = f"""🎬 제목: {show.get('title')}
        📅 개봉일: {show.get('release_date')}
        ⭐ 평점: {show.get('vote_average')}/10
        📖 줄거리: {show.get('overview')}
        """
        
        
        # 포스터만 보여주면 됨
        poster_path = show.get('poster_path')
        poster_url = f"https://image.tmdb.org/t/p/w500{poster_path}"
        # "https://image.tmdb.org/t/p/w500{poster_path}" : 포스터 경로 
        return show_info, poster_url

    except requests.exceptions.RequestException as e:
        return f"API 요청 중 오류 발생: {str(e)}"
    except IndexError:
        return f"영화 정보를 처리하는 중 오류 발생: 검색 결과가 없습니다."

# 챗봇 함수
def show_chatbot(message):
    # 1. TMDB로부터 드라마 정보 받아오기
    show_info = TV(message)
    
    # 드라마 정보를 찾지 못한 경우 그대로 반환
    if "찾을 수 없습니다" in show_info or "오류 발생" in show_info:
        return show_info

    # 튜플 언패킹(Tuple Unpacking)
    # 분리
    show_info, poster_url = show_info

    # 2. AI 모델에 전달할 "프롬프트 구성과 이해"
    prompt = f"""당신은 친근하고 전문적인 TV프로그램 전문가 챗봇입니다. 
다음 TV프로그램 정보를 바탕으로 사용자에게 재미있고 유익한 정보를 제공해주세요.


TV프로그램 정보: {show_info} 

사용자 질문: {message}

사용자들이 ~설명해주세요 혹은 ~알려주세요라고 해도 영화정보를 알려주시면 됩니다.
위 정보를 바탕으로 TV프로그램에 대해 친근하고 상세하게 설명해주세요."""

    try:
        # Ollama 모델 호출 
        # Ollama 모델에 전달하여 응답을 생성합니다. (invoke)
        response = model.invoke(prompt)
        return poster_url, response
    
    except Exception as e:
        return f"챗봇 응답 생성 중 오류 발생: {str(e)}"

# Gradio 인터페이스 생성
iface2 = gr.Interface(
    # 함수만 받습니다
    fn=show_chatbot,
    # example
    inputs=gr.Textbox(lines=10, placeholder="TV프로그램 제목을 입력하세요 (예: 고든램지의 헬스키친)"),
    outputs=[gr.Image(label="포스터", show_label=True),  
             gr.Textbox(label="상세설명", lines=20)
                   
    ],
    title="🎬 TV프로그램 정보 챗봇",
    description="TV프로그램 제목을 입력하면 상세한 정보와 함께 AI가 TV프로그램에 대해 설명해드립니다!",
    # examples=[
    #     ["기생충"],
    #     ["어벤져스가 뭐야?"],
    #     ["타이타닉에 대해 알려줘"],
    #     ["인터스텔라는 재미있어?"],
    #     ["기생충이라는 영화 있어?"],
    #     ["탑건 설명해줘"]
    # ]
)


app = gr.TabbedInterface([iface, iface2], ["영화", "드라마"])


# 서버 실행
# if __name__ == "__main__":
app.launch(server_port=7874, server_name="0.0.0.0", share=True)

* Running on local URL:  http://0.0.0.0:7874
* Running on public URL: https://7c09cbbebc7116c0dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


결과: [{'adult': False, 'backdrop_path': '/pN3eaCl3sqwrerU8UNdp40F2mK0.jpg', 'genre_ids': [878, 12, 14], 'id': 83533, 'original_language': 'en', 'original_title': 'Avatar: Fire and Ash', 'overview': "In the wake of the devastating war against the RDA and the loss of their eldest son, Jake Sully and Neytiri face a new threat on Pandora: the Ash People, a violent and power-hungry Na'vi tribe led by the ruthless Varang. Jake's family must fight for their survival and the future of Pandora in a conflict that pushes them to their emotional and physical limits.", 'popularity': 7.1041, 'poster_path': '/vDdFxksDMkUJjZXd00OKJfwXKre.jpg', 'release_date': '2025-12-17', 'title': '아바타: 불과 재', 'video': False, 'vote_average': 0.0, 'vote_count': 0}]
결과2: {'adult': False, 'backdrop_path': '/pN3eaCl3sqwrerU8UNdp40F2mK0.jpg', 'genre_ids': [878, 12, 14], 'id': 83533, 'original_language': 'en', 'original_title': 'Avatar: Fire and Ash', 'overview': "In the wake of the devastating war against the RDA and the l

Traceback (most recent call last):
  File "c:\Project\.venv\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\Project\.venv\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Project\.venv\lib\site-packages\gradio\blocks.py", line 2228, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "c:\Project\.venv\lib\site-packages\gradio\blocks.py", line 1955, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
  File "c:\Project\.venv\lib\site-packages\gradio\blocks.py", line 1910, in validate_outputs
    raise ValueError(
ValueError: A  function (flim_chatbot) didn't return enough output values (needed: 2, returned: 1).
    Output components:
        [image, textbox]
    Output values returned:
        ["챗봇 응답 생성 중 오류 발생: Ollama call failed with status co

결과: [{'adult': False, 'backdrop_path': '/2F3R4LNKkD1thDxE2SyjGXwWmtR.jpg', 'genre_ids': [27], 'id': 26466, 'original_language': 'en', 'original_title': 'Triangle', 'overview': '친구들과 요트 여행에 오른 싱글맘 제스.  갑작스러운 폭풍을 만나 일행 모두 바다에 표류하지만 운 좋게도 호화 유람선을 발견하고 도움을 청하기 위해 승선한다. 하지만 배 안에는 사람의 흔적만 느껴질 뿐 아무도 보이지 않고 바다 위, 마치 시간이 멈춰버린 듯한 거대한 크루즈 안에서 일행들은 한 명씩 의문의 죽음을 맞게 된다. 끝을 알 수 없이 계속 반복되는 죽음과 공포의 순간, 정해진 운명의 패턴을 바꿔야만 탈출에 성공할 수 있는데...  과연 제스는 반복되는 시간의 고리를 끊고 운명의 시계를 되돌릴 수 있을까?', 'popularity': 5.6923, 'poster_path': '/wObkLgF4XiwuW43dVjBZ50dxZoM.jpg', 'release_date': '2009-10-16', 'title': '트라이앵글', 'video': False, 'vote_average': 6.9, 'vote_count': 2726}, {'adult': False, 'backdrop_path': '/baBarpurV0viPlS6YiDCDeKXdkN.jpg', 'genre_ids': [35, 10749, 80], 'id': 240763, 'original_language': 'ko', 'original_title': '트라이앵글', 'overview': '미모의 재벌가 미망인 `지영`은 스키장에서 우연히 만난 미술 전시 기획 CEO `상우`에게 호감을 갖는다. 젠틀 한데다 재력을 갖추고 있는 상우를 보며 자신의 재산만 노리는 여타 남자들과 다를 거란 믿음에 마음을 열기 시작하는 지영.  하지만 사실 상우는 지갑에 남은 건 4천원뿐인 빈털터리 사기꾼. 그녀가 

Traceback (most recent call last):
  File "c:\Project\.venv\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\Project\.venv\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Project\.venv\lib\site-packages\gradio\blocks.py", line 2228, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "c:\Project\.venv\lib\site-packages\gradio\blocks.py", line 1955, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
  File "c:\Project\.venv\lib\site-packages\gradio\blocks.py", line 1910, in validate_outputs
    raise ValueError(
ValueError: A  function (flim_chatbot) didn't return enough output values (needed: 2, returned: 1).
    Output components:
        [image, textbox]
    Output values returned:
        ["챗봇 응답 생성 중 오류 발생: Ollama call failed with status co

결과: [{'adult': False, 'backdrop_path': '/wwNasWOG4HlH9R6F8ybPJETTPO7.jpg', 'genre_ids': [80, 18, 9648], 'id': 70626, 'origin_country': ['KR'], 'original_language': 'ko', 'original_name': '비밀의 숲', 'overview': '감정을 느끼지 못하는 외톨이 검사 황시목이, 정의롭고 따뜻한 형사 한여진과 함께 검찰 스폰서 살인사건과 그 이면에 숨겨진 진실을 파헤치는 내부 비밀 추적극. 처음엔 검찰 조직 내부의 비리에서 촉발된 것으로 보였던 사건은 범인의 의도도, 향방도 알 수 없는 미궁에 빠지는데...', 'popularity': 28.269, 'poster_path': '/4NCHVr4BiuZzbqoRaLMTbmKIfyR.jpg', 'first_air_date': '2017-06-10', 'name': '비밀의 숲', 'vote_average': 8.292, 'vote_count': 178}]
결과2: {'adult': False, 'backdrop_path': '/wwNasWOG4HlH9R6F8ybPJETTPO7.jpg', 'genre_ids': [80, 18, 9648], 'id': 70626, 'origin_country': ['KR'], 'original_language': 'ko', 'original_name': '비밀의 숲', 'overview': '감정을 느끼지 못하는 외톨이 검사 황시목이, 정의롭고 따뜻한 형사 한여진과 함께 검찰 스폰서 살인사건과 그 이면에 숨겨진 진실을 파헤치는 내부 비밀 추적극. 처음엔 검찰 조직 내부의 비리에서 촉발된 것으로 보였던 사건은 범인의 의도도, 향방도 알 수 없는 미궁에 빠지는데...', 'popularity': 28.269, 'poster_path': '/4NCHVr4BiuZzbqoRaLMTbmKIfyR.jpg', 'first_air_date': '2017-06-10'

Traceback (most recent call last):
  File "c:\Project\.venv\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\Project\.venv\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Project\.venv\lib\site-packages\gradio\blocks.py", line 2228, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "c:\Project\.venv\lib\site-packages\gradio\blocks.py", line 1955, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
  File "c:\Project\.venv\lib\site-packages\gradio\blocks.py", line 1910, in validate_outputs
    raise ValueError(
ValueError: A  function (show_chatbot) didn't return enough output values (needed: 2, returned: 1).
    Output components:
        [image, textbox]
    Output values returned:
        ["챗봇 응답 생성 중 오류 발생: Ollama call failed with status co

In [ ]:
iface.close()